In [18]:
import openai
import os
import random
import time

openai.api_key = os.environ["openaipersonal"]

In [10]:
# nyt puzzle for August 27, 2023

sudoku_str = """_21___4_6
_8792____
__486__27
7____985_
5_92____1
2__7_36__
_7__8236_
892_3__7_
_3___7__8""".split("\n")

row_indices = "abcdefghi"
col_indices = "БВГДЕЖЗИК"
square_indices = "αβγδεζηθι"

new_sudoku_str = []
for row in range(9):
    new_row = ""
    for col in range(9): 
        new_row += f"{row_indices[row]}{col_indices[col]}{square_indices[(row//3)*3 + col//3]}: {sudoku_str[row][col]} | "
    new_row = "".join(new_row)
    new_sudoku_str.append(new_row)
    if row in [2, 5]:
        new_sudoku_str.append("====")
    else:
        new_sudoku_str.append("----")

grid = "\n".join(new_sudoku_str)

grid += "\n Recently tested groups: a, b, c, Б, В, Г, α, β, γ, δ."
print(len(grid))
grid

837


'aБα: _ | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | \n----\nbБα: _ | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | \n----\ncБα: _ | cВα: _ | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: _ | cЗγ: _ | cИγ: 2 | cКγ: 7 | \n====\ndБδ: 7 | dВδ: _ | dГδ: _ | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | \n----\neБδ: 5 | eВδ: _ | eГδ: 9 | eДε: 2 | eЕε: _ | eЖε: _ | eЗζ: _ | eИζ: _ | eКζ: 1 | \n----\nfБδ: 2 | fВδ: _ | fГδ: _ | fДε: 7 | fЕε: _ | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | \n====\ngБη: _ | gВη: 7 | gГη: _ | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | \n----\nhБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | \n----\niБη: _ | iВη: 3 | iГη: _ | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | \n----\n Recently tested groups: a, b, c, Б, В, Г, α, β, γ, δ.'

In [11]:
import re

def find_latest_grid(text):
    pattern = (r"(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}"
               r"\n={4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}"
               r"\n={4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}\n-{4}\n?(?:[a-i][Б-К][α-ι]: (?:\d|_)(?: \| )?){9}")

    matches = list(re.finditer(pattern, text))
    
    if matches:
        return matches[-1].group(0)  # Returns the latest (last) match
    return None

In [12]:
row_indices = "abcdefghi"
col_indices = "БВГДЕЖЗИК"
square_indices = "αβγδεζηθι"

def process_str_grid(sudoku_grid):
    sudoku_grid = find_latest_grid(sudoku_grid)
    rows = [row for row in sudoku_grid.strip().split('\n') if not row.startswith('-') and not row.startswith('=')]
    processed_grid = [[int(cell.split(': ')[1]) if cell.split(': ')[1] != '_' else 0 for cell in row.rstrip(' |').split(' | ')] for row in rows]
    return processed_grid

def is_valid_sudoku(grid):
    # Check rows
    for row in range(9):
        numbers_seen = set()
        for col in range(9):
            num = grid[row][col]
            if num != 0:
                if num in numbers_seen:
                    return False, f'Contradiction at row {row_indices[row]}'
                numbers_seen.add(num)

    # Check columns
    for col in range(9):
        numbers_seen = set()
        for row in range(9):
            num = grid[row][col]
            if num != 0:
                if num in numbers_seen:
                    return False, f'Contradiction at column {col_indices[col]}'
                numbers_seen.add(num)

    # Check 3x3 squares
    for square in range(9):
        numbers_seen = set()
        start_row = (square // 3) * 3
        start_col = (square % 3) * 3
        for row in range(start_row, start_row + 3):
            for col in range(start_col, start_col + 3):
                num = grid[row][col]
                if num != 0:
                    if num in numbers_seen:
                        return False, f'Contradiction at square {square_indices[square]}'
                    numbers_seen.add(num)

    return True, 'No contradiction'

In [19]:
system_prompt = """You are a very sophisticated LLM. 
Your current task is to train a baby LLM in precise, acurate, logical thinking.
For this exercise, you are demonstrating the logical steps to solve a sudoku.
It is important that you always write out the logical steps in full, because the baby LLM you are training needs to understand them correctly.
It is very important that you don't skip steps. 
It is absolutely crucial that you first list the predicates, and only then the conclusions. 
I.e. always say, "Because of X and Y, therefore Z" and never say "Z, because X and Y".
"""

def generate_user_prompt(grid):
#     groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι']
#     sampled_elements = random.sample(groups, 8)
#     print("generating prompt with groups ", sampled_elements)
#     str_groups = ", ".join(sampled_elements)
    
    user_prompt = f"""Here is the sudoku that needs to be solved:

####
{grid}
####

It is written in a rather peculiar way, to aid the understanding of the baby AI you're teaching.

There are added indices to each square, indicating the row, the column, and the 3x3 square each square belongs to.
Rows have an index from a, b, c, d, e, f, g, h, i, 
columns have an index from Б, В, Г, Д, Е, Ж, З, И, К, 
3x3 squares have an index from α, β, γ, δ, ε, ζ, η, θ, ι. 
Empty squares are represented as "_".
You call each column, row, and 3x3 square a "group". 
Each group should have the numbers from 1 to 9 exactly. 
Every group has exactly 9 squares.  
A group CANNOT contain the same number twice. 
A group MUST contain every number from 1 to 9.

Using this index, you can tell exactly to which groups a square belongs.
Example: gБη belongs to groups g, Б, η. 
Example: dЕε belongs to groups d, Е, and ε. 

You can also tell which squares belong in a certain group: all who contain the group in their index.
Example: group a contains squares aБα, aВα, aГα, aДβ, aЕβ, aЖβ, aЗγ, aИγ, aКγ. Therefore, group a contains the numbers ...
Example: group К contains squares aКγ, bКγ, cКγ, dКζ, eКζ, fКζ, gКι, hКι, iКι. Therefore, group К contains the numbers ...

Additionally: groups intersect. The squares which have the indices of two groups, belong to both groups.
Example: group a intersects group α in the squares aБα, aВα, aГα.
Example: column Е intersects row f only in the square fЕε.

Please solve it with the following steps:

1) Select 8 groups from all groups in the sudoku. 
Alternate between row, column, 3x3 square. 
Choose the groups that contain a low, but non-zero number of empty squares.
Try not to choose only groups from the start of the list, sometimes choose from the end as well.
You CANNOT choose any groups mentioned in the recently tested groups list.
For each of the 8 groups you chose, name the cells they contain by index.

Reminder that a group contains all cells that have the group name as part of their index.
Be concise. 
Examples:
"Group a: aБα, aВα, aГα, aДβ, aЕβ, aЖβ, aЗγ, aИγ, aКγ"
"Group Б: aБα, bБα, cБα, dБδ, eБδ, fБδ, gБη, hБη, iБη"
"Group α: aБα, aВα, aГα, bБα, bВα, bГα, cБα, cВα, cГα"

1.1) For each of the selected groups above:
* name the numbers they contain, with the respective index;
* the numbers remaining
* which cells are empty (by index)
* how many cells are how many are filled in and how many are empty
Your job here is to check the list you created in the previous step against the sudoku grid above.

Example:
"Group θ -  numbers gДθ: 1, gЕθ: _, gЖθ: _, hДθ: 7, hЕθ: _, hЖθ: 4, iДθ: _, iЕθ: 3, iЖθ: 8;
Missing the numbers - 2, 5, 6, 9.
The empty squares in group θ are gЕθ, gЖθ, hЕθ, iДθ.
The filled in numbers are 5, the empty squares are 4."

Example:
"Group δ - contains the numbers dБδ: 2, dВδ: _, dГδ: _, eБδ: 8, eВδ: 9, eГδ: _, fБδ: 5, fВδ: 4, fГδ: 3; 
Missing the numbers - 1, 6, 7. 
The empty squares in group δ are dВδ, dГδ, eГδ.
The filled in numbers are 6, the empty squares are 3."

2) Pick and target one specific group from the ones you've chosen. 
We will call it the "target group". Try to pick the one that has the least number of empty squares, but that has at least 1 empty square.  
Name all of it's empty squares, and the numbers we need to fill them with. 
Name all groups that intersect with its empty squares. 
Here, it is very important to be very precise when listing the intersecting groups - list them all, but be careful not to list any others.
Remember, if our target group is a row, it cannot intersect with other rows, if it's a column, with other columns, and if it's a square, with other squares.

Example:
"Among our chosen groups, the groups with the smallest non-zero number of empty squares have 2 empty squares.
Those are groups g and ζ. 
Let's select as our target group, g.
It contains the empty squares gГη, gЗι and intersects groups Г, η, З, ι."

Example:
"Among our chosen groups, the groups with the smallest non-zero number of empty squares have 4 empty squares.
Those are groups Б, В, f, i. 
Let's select as our target group, Б.
It contains the empty squares aБα, bБα, cБα, iБη and intersects groups a, b, c, i, α, η."

3) Now we have the missing numbers from our target group, the empty squares and the intersecting groups.
We want to know what numbers we can fill in in our target group. 
We will exclude some possibilities for some squares.

Do the following:
* list the missing numbers and in which squares we need to fill them in, as well as the intersecting groups.
* for each intersecting group:
    ** list all the values it contains in full notation
    ** list all the values it contains in simple notation, then write which of our empty squares it intersects
    ** then write if it excludes any value for the empty squares in our target group.

Example: 
"We need to fill in numbers 1, 2, 7 in squares dВδ, eБδ, eГδ. The intersecting groups are В, e, d.
Intersecting group В: aВα: 9, bВα: 1, cВα: _, dВδ: _, eВδ: _, fВδ: _, gВη: 8, hВη: _, iВη: 2;
Intersecting group В contains 1, 2, 8, 9.
It intersects square dВδ, therefore square dВδ cannot be 1, 2, 8 or 9.

Intersecting group e: eБδ: _, eВδ: 3, eГδ: _, eДε: _, eЕε: _, eЖε: _, eЗζ: 7, eИζ: _, eКζ: 4;
Intersecting group e contains 3, 4, 7.
It intersects squares eБδ and eГδ, therefore squares eБδ and eГδ cannot be 3, 4 or 7.

Intersecting group d: dБδ: 5, dВδ: _, dГδ: _, dДε: _, dЕε: 6, dЖε: _, dЗζ: 8, dИζ: _, dКζ: _;
Intersecting group d contains 5, 6, 8.
It contains square dВδ, therefore square dВδ cannot be 5, 6, 8."

Make sure to mention every intersecting group from the previous step exactly once, and in order.

3.1) For every empty square in our target group, provide a union of all the excluded numbers listed in step 3.

Example:
"Referring to step 3, square dВδ cannot be 1, 2, 8, 9, 5, 6."

4) Combine all your observations. 
For each empty square in the target group, what are the numbers we need to fill in in our target group?
We will call these the "initial possibilities".
Initially, they are the same for every empty square in our target group.
For each empty square in the target group, what are the numbers we've excluded in step 3?
We will call these the "excluded possibilities".
They are the union of all numbers we said are not possible for the respective empty square in step 3.
Remember, every empty square should be mentioned twice in the intersecting groups in step 3.
Finally, for every square, what are the final possibilities?
Those are calculated by subtracting the two sets : "initial possibilities" - "final possibilities".

Afterwards, if the final possibilities for any given empty square are only 1 number, we can fill it in!


Example:
Square X: initial possibilities [1, 5, 9]. Referring to step 3, the excluded possibilities for square X are [1, 3, 4, 5, 8]. 
Therefore, the final possibilities for X are [9].
Square Y: initial possibilities [1, 5, 9]. Referring to step 3, the excluded possibilities for square Y are [1, 2, 3, 6, 7].
Therefore, the final possibilities for Y are [5, 9].

Important - not always will you be able fill in a number in your target group. 
That is fine, if so, return to step 2 and choose the next best target group.
You are demonstrating the logic.

Important - if you have exhausted all possibilities for a given square, DO NOT attempt to fill it in, you've made a mistake somewhere. 
Ignore this square and continue with the steps.

Likewise, if you find that there is the same value remaining for two squares, you have made a mistake. Ignore them and continue.
You are demonstrating a robust process of thinking.

5) Exactly reproduce the "recently tested groups" you got in your instructions.
Afterwards, write them out again, except begin with the target group we chose in the first place, and drop the last one from the list.
The list is exactly the same, except we add one group in the front and remove one group from the end of the list.

For example, if you were given the list:
"Recently tested groups (old): i, З, Е, ζ, К, Б, α, c, g, β.", and you tested group Ж, you should write:
"Recently tested groups: Ж, i, З, Е, ζ, К, Б, α, c, g."

6) If you have deduced for certain what an empty square must contain, fill it in.
Write the sudoku grid with the new number filled in, in exactly the same format, including coordinates.
Otherwise, write the sudoku grid as received from the instructions with no new numbers filled in.
In both cases, there MUST be a entire sudoku grid, otherwise the baby AI will not understand our process.

After the sudoku grid, again repeat the recently tested groups from step 5:
"Recently tested groups: ..." 

Do not say anything else besides the sudoku grid and the recently tested groups in this step.
"""
    return user_prompt

In [20]:
answers = []
grids = [grid]
new_grid = grid

def take_steps(maximum_steps=50, current_step=0):
    if current_step == maximum_steps:
        print("maximum steps reached, ending process :)")
        return
    
    time.sleep(60) # because gpt is prissy otherwise
    print(f"now on step {current_step}, current time is {time.strftime('%H:%M')}")
    user_prompt = generate_user_prompt(grids[-1])
    
    try:
        response = openai.ChatCompletion.create(
                      model="gpt-4",
                      messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": user_prompt},
                        ],
                        temperature=0,
                    )

        answer = response['choices'][0]['message']['content']
        answers.append(answer)
        new_grid = answer[-842:]
        
        print("\n\n", new_grid, "\n\n")
        grids.append(new_grid)
        print("remaining empty squares: ", new_grid.count("_"))
        
        try:
            processed_grid = process_str_grid(new_grid)
            correct, problem = is_valid_sudoku(processed_grid)

        except:
            print("could not find/process grid")
            return
            
        if correct:
            print("it's a valid grid")
        else:
            print(f"invalid grid, {problem}")
            return
            
        if not new_grid.count("_"):
            print("🎉🎊🥳🚀✨ Complete! ✨🚀🥳🎊🎉")
            return
        
        take_steps(maximum_steps, current_step+1)
    
    except Exception as e:
        print(f"following error encountered calling gpt...", e, "\n")
        # time.sleep(180)  # no, stop, you're having too much fun, reee
        take_steps(maximum_steps, current_step)
        
take_steps(maximum_steps = 50)

now on step 0, current time is 14:14


 
aБα: _ | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | 
----
bБα: _ | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: _ | cВα: _ | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: _ | cЗγ: _ | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: _ | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: _ | eГδ: 9 | eДε: 2 | eЕε: _ | eЖε: _ | eЗζ: _ | eИζ: _ | eКζ: 1 | 
----
fБδ: 2 | fВδ: _ | fГδ: _ | fДε: 7 | fЕε: _ | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: _ | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: _ | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | 
----
Recently tested groups: h, a, b, c, Б, В, Г, α, β, γ.
#### 


remaining empty squares:  43
it's a valid grid
now on step 1, current time is 14:18


 
aБα: _ | aВα: 2 | aГα: 1 |



 
aБα: _ | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | 
----
bБα: _ | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: _ | cВα: _ | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: _ | cЗγ: _ | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: _ | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: _ | eГδ: 9 | eДε: 2 | eЕε: _ | eЖε: _ | eЗζ: 7 | eИζ: _ | eКζ: 1 | 
----
fБδ: 2 | fВδ: 1 | fГδ: _ | fДε: 7 | fЕε: _ | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: 5 | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: 6 | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | 
----
Recently tested groups: В, f, В, Г, ζ, e, g, d, h, a.
#### 


remaining empty squares:  39
it's a valid grid
now on step 9, current time is 15:00
following error encountered calling gpt... Bad gateway. {"error":{"

now on step 17, current time is 15:42
following error encountered calling gpt... Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600) 

now on step 17, current time is 15:53


 ####
aБα: _ | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | 
----
bБα: _ | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: _ | cВα: 5 | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: 1 | cЗγ: 9 | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: 3 | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: _ | eГδ: 9 | eДε: 2 | eЕε: 4 | eЖε: _ | eЗζ: 7 | eИζ: _ | eКζ: 1 | 
----
fБδ: 2 | fВδ: 1 | fГδ: 8 | fДε: 7 | fЕε: _ | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: 5 | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: 6 | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι:

now on step 26, current time is 16:34


 ####
aБα: 9 | aВα: 2 | aГα: 1 | aДβ: _ | aЕβ: _ | aЖβ: _ | aЗγ: 4 | aИγ: _ | aКγ: 6 | 
----
bБα: 6 | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: 3 | cВα: 5 | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: 1 | cЗγ: 9 | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: 3 | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: 6 | eГδ: 9 | eДε: 2 | eЕε: 4 | eЖε: 8 | eЗζ: 7 | eИζ: 3 | eКζ: 1 | 
----
fБδ: 2 | fВδ: 1 | fГδ: 8 | fДε: 7 | fЕε: 5 | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: 5 | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: 6 | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | 
----

Recently tested groups: Д, e, f, Б, З, α, ζ, В, α, Г. 


remaining empty squares:  26
it's a valid grid
now on step 27, current time is 16:40


 ####
aБα: 9 | aВα: 2 | aГ

now on step 35, current time is 20:45
following error encountered calling gpt... Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600) 

now on step 35, current time is 20:56


 aБα: 9 | aВα: 2 | aГα: 1 | aДβ: 3 | aЕβ: 7 | aЖβ: 5 | aЗγ: 4 | aИγ: 8 | aКγ: 6 | 
----
bБα: 6 | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: _ | bЗγ: _ | bИγ: _ | bКγ: _ | 
----
cБα: 3 | cВα: 5 | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: 1 | cЗγ: 9 | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: 3 | dДε: _ | dЕε: _ | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: 6 | eГδ: 9 | eДε: 2 | eЕε: 4 | eЖε: 8 | eЗζ: 7 | eИζ: 3 | eКζ: 1 | 
----
fБδ: 2 | fВδ: 1 | fГδ: 8 | fДε: 7 | fЕε: 5 | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: 5 | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: 6 | iДθ: _ | iЕθ: _ | iЖθ: 7 | iЗι: _ | iИι: _ | 

now on step 44, current time is 21:42


 α: 9 | aВα: 2 | aГα: 1 | aДβ: 3 | aЕβ: 7 | aЖβ: 5 | aЗγ: 4 | aИγ: 8 | aКγ: 6 | 
----
bБα: 6 | bВα: 8 | bГα: 7 | bДβ: 9 | bЕβ: 2 | bЖβ: 4 | bЗγ: 1 | bИγ: _ | bКγ: _ | 
----
cБα: 3 | cВα: 5 | cГα: 4 | cДβ: 8 | cЕβ: 6 | cЖβ: 1 | cЗγ: 9 | cИγ: 2 | cКγ: 7 | 
====
dБδ: 7 | dВδ: _ | dГδ: 3 | dДε: 6 | dЕε: 1 | dЖε: 9 | dЗζ: 8 | dИζ: 5 | dКζ: _ | 
----
eБδ: 5 | eВδ: 6 | eГδ: 9 | eДε: 2 | eЕε: 4 | eЖε: 8 | eЗζ: 7 | eИζ: 3 | eКζ: 1 | 
----
fБδ: 2 | fВδ: 1 | fГδ: 8 | fДε: 7 | fЕε: 5 | fЖε: 3 | fЗζ: 6 | fИζ: _ | fКζ: _ | 
====
gБη: _ | gВη: 7 | gГη: 5 | gДθ: _ | gЕθ: 8 | gЖθ: 2 | gЗι: 3 | gИι: 6 | gКι: _ | 
----
hБη: 8 | hВη: 9 | hГη: 2 | hДθ: _ | hЕθ: 3 | hЖθ: _ | hЗι: _ | hИι: 7 | hКι: _ | 
----
iБη: _ | iВη: 3 | iГη: 6 | iДθ: _ | iЕθ: 9 | iЖθ: 7 | iЗι: _ | iИι: _ | iКι: 8 | 
----

Recently tested groups: b, ι, Б, В, ε, Ж, γ, Е, f.
####
#### 


remaining empty squares:  17
could not find/process grid


In [ ]:
# for debug purposes 

print(generate_user_prompt(grids[-2]))

In [ ]:
# for debug purposes

print(len(grids))
print(grids[-2])

process_str_grid(grids[-1])
# processed_grid = process_str_grid(new_grid)
# correct, problem = is_valid_sudoku(processed_grid)

In [ ]:
# for debug purposes

print(len(answers))
print(answers[-1])